### 简历项目3：Ollama构建本地轻量级RAG系统
#### 1. 项目简介

#### 2. 数据集 & RAG系统
&emsp;&emsp;RAG基础阅读：[16种RAG方案一站式攻略](https://blog.csdn.net/l01011_/article/details/149929414)、[七大Agentic RAG框架技术解析](https://zhuanlan.zhihu.com/p/19229901774) <br>
&emsp;&emsp;[RAGEval数据集文献](https://arxiv.org/abs/2408.01262): Scenario Specific RAG Evaluation Dataset Generation Framework  <br>
&emsp;&emsp;[LightRAG文献](https://arxiv.org/abs/2410.05779)：Simple and Fast Retrieval-Augmented Generation <br>
&emsp;&emsp;本地配置：Ollama推理Qwen3-4B-Q8_0.gguf大模型和bge-m3-FP16.gguf嵌入模型，可以在魔搭社区上提前下载好再使用Modelfile配置一下，vllm推理bge-reranker重排序模型，在运行LightRAG之前在终端开启以上服务。<br>

#### 3. LightRAG解读
<div align="center">
   <img src="./pics/rp3_rag_1.png" width="1000" />
</div>

&emsp;&emsp;参考阅读:[深度解析：LightRAG知识图谱实现于应用分析](https://mp.weixin.qq.com/s/scZ1WZRS3v1QBOYV_KYFTw)、

&emsp;&emsp;知识图谱是 LightRAG 区别于传统 RAG 系统的核心特性，它不仅将文档转换为向量，还构建了一个结构化的知识网络，让系统能够理解实体之间的关系，进行推理和发现。
#### 3.1 知识图谱构建流程
&emsp;&emsp;lightrag.py中的apipeline_process_enqueue_documents函数是LightRAG的文档处理管道，负责将待处理文档转换为知识图谱，主要功能如下：<br>
##### 3.1.1 锁机制
&emsp;&emsp;使用分布式锁确保同一时间只有一个进程处理队列，收集三种状态的文档(processing,failed,pending)，如果已有进程在处理，设置request_pending标志并返回。<br>
##### 3.1.2 文档收集与状态初始化
&emsp;&emsp;合并待处理文档，初始化管道状态（作业名称、开始时间、进度等）
##### 3.1.3 文档主循环
**阶段1：文档分块与存储**<br>
两种分块策略类型：<br>
&emsp;&emsp;1、基于Token的分块（默认），按照token数量分割文档，默认块的大小为1200tokens，重叠大小为100tokens。<br>
&emsp;&emsp;2、基于字符的分块，按照指定字符分割文档，支持仅按字符分割或结合token大小限制。**参数说明**：<font color='red'>**split_by_character: str | None**</font>，类型: str | None，说明: 指定用来分割文本的字符，如果不是 None，会先按照这个字符分割文本，然后再根据 token 限制进一步分割（除非设置了 split_by_character_only=True）；<font color='red'>**split_by_character_only: bool**</font>，类型: bool，说明:如果为 True，仅按 split_by_character 指定的字符分割，不考虑 token 限制，如果为 False，会在按字符分割后，再按 token 大小限制进行二次分割。<br>
&emsp;&emsp;**阶段2：实体关系提取**<br>
&emsp;&emsp;**阶段3：合并到知识图谱**<br>
##### 3.1.4 并发控制
##### 3.1.5 错误处理与状态管理
##### 3.1.6 循环处理与请求队列
##### 3.1.7 清理与状态重置
**处理流程总结**<br>
&emsp;待处理文档 → 文档分块 → 存储文本块/向量块 → 提取实体关系 → 合并到知识图谱 → 更新文档状态<br>
&emsp;&emsp;↓<br>
&emsp;并发控制（信号量）<br>
&emsp;&emsp;↓<br>
&emsp;错误处理与状态管理<br>
&emsp;&emsp;↓<br>
&emsp;循环检查新文档<br>

#### 4. 面试问答
(1) 有没有了解过GraphRAG？<br>
(2) 